In [2]:
#!/usr/bin/python
import os, sys
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
from io import StringIO
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from bokeh.plotting import figure, output_notebook, show


In [3]:

def powerlaw(x,a,b,c):
    return a*x**b+c

def f5(x,a,b,c,d,g,h):
    return a*x**5+b*x**4+c*x**3+d*x**2+g*x+h

def f3(x,a,b,c,d):
    return a*x**3+b*x**2+c*x+d


def VowelToConsRatio(name):
    
    words = sum(c.isalpha() for c in name)
    spaces = sum(c.isspace() for c in name)
    others = len(name) - words - spaces
    vowels = sum(map(name.lower().count, "aeiou"))
    consonents = words - vowels


    return float(vowels)/float(consonents), float(spaces)/float(consonents), float(others)/float(consonents)

def CharCount(name):
    
    words = sum(c.isalpha() for c in name)
    spaces = sum(c.isspace() for c in name)
    others = len(name) - words - spaces
    vowels = sum(map(name.lower().count, "aeiou"))
    consonents = words - vowels


    return int(consonents), int(vowels), int(spaces), int(others)

def MergeCount(totalCount, thisCount):
    totalCount2=totalCount+thisCount
    return totalCount2



def LatLongClassMaker(coor):
    lat,long=coor
    if lat<30:
        if long>-90:
            return 1 #florida
        elif long < -120:
            return 2 #hawaii
        else:
            return 3 #texas
    elif lat >50:
        return 4 #alaska
    elif long> -80 and lat < 40:
        return 5 #south east
    elif long> -80 and lat >40:
        return 6 #new england
    elif long< -80:
        return int((50-lat)/20.*5)*int((125+long)/45.*9)+7
    else:
        return 0
    #worst mapping ever-- ignores new england and city diversity
    return 0




In [10]:
ratio=[]
coords=[]
charcountlist=[]
file = open("airports-extended.dat.txt","r")
for line in file:
    line=re.split(',',line);

    if line[3] == "\"United States\"":
        #thisratioV, thisratioS, thisratioO=VowelToConsRatio(line[1]);
        #ratio.append([thisratioV,thisratioS,thisratioO])
        charcountlist.append(np.array([CharCount(line[1])]))
        coords.append(np.array([float(line[6]),float(line[7])]))


#ratioarray=np.empty([len(ratio),3])
coordsarray=np.empty([len(coords),2])
regionarray=np.empty([len(coords)])
charcountarray=np.empty([len(coords),4])

#for i, rat in enumerate(ratio):
#    ratioarray[i,:]=rat
for i, charct in enumerate(charcountlist):
    charcountarray[i,:]=charct
for i,coor in enumerate(coords):
    coordsarray[i,:]=coor
#    outputcoor=LatLongClassMaker(coor)
#    regionarray[i]=outputcoor




In [12]:
np.shape(charcountarray)

(2294, 4)

In [13]:
testphrases=["Ufda!", "I just dont know about that", "Howdy, pardner", "Where are we?", "I thank whatever gods may be for my unconquerable soul.", "I welcome our new robot overlords", "l33t h4k3rs", "I know you I walked with you once upon a dream", "These stories dont mean anything if you have no one to tell them to", "Baby you have the sort of hands that rip me apart","Multi-messenger astronomy", "Rainbow flag", "I prefer They or He?", "numerical relativity", "LIGO", "scalar field", "Osculating Orbits", "Monte-Carlo Simulation", "Data Analysis", "Data Science", "Parallelization", "Paralyzation", "Partial disability", "Non-epileptic seizures", "Wednesday Lunch", "Tuesday Lunch", "Thursday Lunch", "Guild Wars", "Elvenar", "Good Apple"]


colleaguephrases=["general relativity", "black hole", "loop quantum gravity",  "quantization", "space-time", "Hamiltonian constraint", "Ashtekar", "LiSA", "LIGO", "group", "white hole", "scalar field", "numerical relativity", "cosmology", "diffeomorphism", "continuum limit", "David Berger", "Reisner-Nordstrom", "black hole spacetime", "initial data", "interpretation of quantum mechanics"]



In [5]:
xphrases=[]
for phrase in testphrases:
    print(phrase)
    vtc, stc, otc = VowelToConsRatio(phrase)
    xphrases.append([vtc,stc,otc])
xcolleague=[]
for phrase in colleaguephrases:
    print(phrase)
    vtc, stc, otc = VowelToConsRatio(phrase)
    xcolleague.append([vtc,stc,otc])

print(xphrases)




Ufda!
I just dont know about that
Howdy, pardner
Where are we?
I thank whatever gods may be for my unconquerable soul.
I welcome our new robot overlords
l33t h4k3rs
I know you I walked with you once upon a dream
These stories dont mean anything if you have no one to tell them to
Baby you have the sort of hands that rip me apart
Multi-messenger astronomy
Rainbow flag
I prefer They or He?
numerical relativity
LIGO
scalar field
Osculating Orbits
Monte-Carlo Simulation
Data Analysis
Data Science
Parallelization
Paralyzation
Partial disability
Non-epileptic seizures
Wednesday Lunch
Tuesday Lunch
Thursday Lunch
Guild Wars
Elvenar
Good Apple
general relativity
black hole
loop quantum gravity
quantization
space-time
Hamiltonian constraint
Ashtekar
LiSA
LIGO
group
white hole
scalar field
numerical relativity
cosmology
diffeomorphism
continuum limit
David Berger
Reisner-Nordstrom
black hole spacetime
initial data
interpretation of quantum mechanics
[[1.0, 0.0, 0.5], [0.5714285714285714, 0.357142

In [14]:
xtrain,xtest,ytrain,ytest=train_test_split(charcountarray,regionarray,test_size=0.5,shuffle=True, random_state=False)



In [15]:
trainshp=np.shape(ytrain)
testshp=np.shape(ytest)
trainshp[0]

1147

In [16]:
ytrain_flat = ytrain.flatten()
ytest_flat=ytest.flatten()

In [17]:
hidden_layers=20 #20 lines of division is my biased theory
hidden_size=3 #enough to draw a line with an offset I think, plus an little variance

In [18]:
sess = None

def reset_vars():
    sess.run(tf.global_variables_initializer())

def reset_tf():
    global sess
    if sess:
        sess.close()
    tf.reset_default_graph()
    sess = tf.Session()

In [19]:
len(xtrain)

1147

In [20]:
reset_tf()

In [45]:
%%bash
git add neuralnetwork2.ipynb
git commit neuralnetwork2.ipynb -m "second attempt at neural network, needlessly complicated, going back to first attempt"

[master 1e95b68] second attempt at neural network, needlessly complicated, going back to first attempt
 1 file changed, 491 insertions(+)
 create mode 100644 neuralnetwork2.ipynb


In [21]:
x=xtrain

In [40]:
def make_lstm(x, lstm_init_value, n_chars, lstm_size, n_layers):
    # LSTM
    lstm = tf.contrib.rnn.MultiRNNCell(
        [tf.contrib.rnn.BasicLSTMCell(lstm_size, forget_bias=1.0, state_is_tuple=False)
         for _ in np.xrange(n_layers)],
        state_is_tuple=False)

    # Iteratively compute output of recurrent network
    out, lstm_new_state = tf.nn.dynamic_rnn(lstm, x, initial_state=lstm_init_value, dtype=tf.float32)

    # Linear activation (FC layer on top of the LSTM net)
    out_reshaped = tf.reshape(out, [-1, lstm_size])
    y = tf.layers.dense(out_reshaped, n_chars, activation=None)
    
    return y, tf.shape(out), lstm_new_state

In [41]:
n_layers = 4
n_labels = max(max(ytrain),max(ytest))
lstm_size = 16
n_chars=n_labels #borrowing someone else's code
time_steps = 100

In [42]:
reset_tf()

x = tf.placeholder(tf.int32, shape=(None, None), name="x")
y_true = tf.placeholder(tf.int32, (None, None))
lstm_init_value = tf.placeholder(tf.float32, shape=(None, n_layers*2*lstm_size),
                                 name="lstm_init_value")

In [43]:
x_enc = tf.one_hot(xtrain.flatten(), depth=n_labels)
y_true_enc = tf.one_hot(ytrain.flatten(), depth=n_labels)

In [44]:
y_pred, out_shape, lstm_new_state = make_lstm(x_enc, lstm_init_value, n_labels, lstm_size, n_layers)

AttributeError: module 'numpy' has no attribute 'xrange'

In [ ]:
final_out = tf.reshape(tf.nn.softmax(y_pred), 
                       (out_shape[0], out_shape[1], n_chars))